In [1]:
from pyspark.sql import SparkSession

# Create a local Spark session
spark = SparkSession.builder \
    .appName("Homework") \
    .master("local[*]") \
    .getOrCreate()

# Check Spark version
spark.version

25/03/09 22:15:54 WARN Utils: Your hostname, Abduls-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.18.9 instead (on interface en0)
25/03/09 22:15:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/09 22:15:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'3.5.5'

In [2]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-09 22:16:02--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.112, 3.164.82.160, 3.164.82.197, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M   361KB/s    in 3m 0s   

2025-03-09 22:19:03 (350 KB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [3]:
# Read the parquet file
df = spark.read.parquet("yellow_tripdata_2024-10.parquet")

# Repartition to 4 partitions
df_repartitioned = df.repartition(4)

# Save as parquet
df_repartitioned.write.mode("overwrite").parquet("yellow_repartitioned.parquet")

In [4]:
from pyspark.sql.functions import to_date, col

# Convert pickup datetime to date format
df_with_date = df.withColumn("pickup_date", to_date("tpep_pickup_datetime"))

# Filter for October 15th
oct_15_trips = df_with_date.filter(col("pickup_date") == "2024-10-15")

# Count the records
count = oct_15_trips.count()
print(f"Number of trips on October 15th: {count}")

Number of trips on October 15th: 128893


In [8]:
!wget wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

Prepended http:// to 'wget'
--2025-03-09 22:32:31--  http://wget/
failed: nodename nor servname provided, or not known.
wget: unable to resolve host address ‘wget’
--2025-03-09 22:32:31--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
3.164.82.160, 3.164.82.197, 3.164.82.40, ...ci6vzurychx.cloudfront.net)... 
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-03-09 22:32:32 (980 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]

FINISHED --2025-03-09 22:32:32--
Total wall clock time: 0.8s
Downloaded: 1 files, 12K in 0s (980 MB/s)


In [10]:
# Load zone lookup data
zones_df = spark.read.csv("taxi_zone_lookup.csv", header=True)
zones_df.createOrReplaceTempView("zones")

# Create temp view for taxi data
df.createOrReplaceTempView("trips")

# SQL query to find least frequent pickup zone
query = """
SELECT z.Zone, COUNT(*) AS pickup_count
FROM trips t
JOIN zones z ON t.PULocationID = z.LocationID
GROUP BY z.Zone
ORDER BY pickup_count ASC
"""

least_frequent = spark.sql(query)
least_frequent.show()

+--------------------+------------+
|                Zone|pickup_count|
+--------------------+------------+
|Governor's Island...|           1|
|       Rikers Island|           2|
|       Arden Heights|           2|
|         Jamaica Bay|           3|
| Green-Wood Cemetery|           3|
|Charleston/Totten...|           4|
|   Rossville/Woodrow|           4|
|       West Brighton|           4|
|Eltingville/Annad...|           4|
|       Port Richmond|           4|
|         Great Kills|           6|
|        Crotona Park|           6|
|Heartland Village...|           7|
|     Mariners Harbor|           7|
|Saint George/New ...|           9|
|             Oakwood|           9|
|       Broad Channel|          10|
|New Dorp/Midland ...|          10|
|         Westerleigh|          12|
|     Pelham Bay Park|          12|
+--------------------+------------+
only showing top 20 rows

